In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import mwparserfromhell
import urllib.request
import json
import re
import pandas as pd
from tqdm import tqdm
import pickle


In [2]:
def get_race(tokens, i):
    regex2 = "^[a-zA-z-' ]+[a-zA-Z ]+"
    
    regex_expressions = re.findall(regex2, tokens[i+1])
    
    # join into word
    
    race = " ".join(regex_expressions).strip()
    
    return race

def get_affiliation(tokens, i):
    regex3 = '[a-zA-z ]+'
    
    to_string = ' '.join(tokens[i:])
    regex_expressions = re.findall(regex3, to_string)
    regex_expressions = [k.strip() for k in regex_expressions]
    
    if "Horde" in regex_expressions:
        affiliation = "Horde"
    elif "Alliance" in regex_expressions:
        affiliation = 'Alliance'
    else:
        affiliation = 'Other'
        
    return affiliation

In [3]:
def get_race_and_affiliation(text):

    regex1 = '[a-zA-Z][^A-Z]*'
    
    tokens_race = text.split("\n")
    tokens_affiliation = re.findall(regex1, text.replace("\n",""))

    race = 0
    affiliation = 0

    for i, word in enumerate(tokens_race):
        if 'Race' in word:
            race = get_race(tokens_race,i)
    for i, word in enumerate(tokens_affiliation):
        if 'Affiliation' in word:
            affiliation = get_affiliation(tokens_affiliation,i)
    
    return race, affiliation


In [4]:
with open('df.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [5]:
# url = "https://wowpedia.fandom.com/wiki/Illidan_Stormrage"
# url = "https://wowpedia.fandom.com/wiki/Thrall"
# url = "https://wowpedia.fandom.com/wiki/Ragnaros"
# url = "https://wowpedia.fandom.com/wiki/Deathwing"
# url = 'https://wowpedia.fandom.com/wiki/Velen'
# url = 'https://wowpedia.fandom.com/wiki/Akama'
# url = 'https://wowpedia.fandom.com/wiki/Thargas_Anvilmar'
# url = 'https://wowpedia.fandom.com/wiki/Garrosh_Hellscream'
# url = 'https://wowpedia.fandom.com/wiki/Leoroxx'
# url = 'https://wowpedia.fandom.com/wiki/Dagran_Thaurissan'
# url = "https://wowpedia.fandom.com/wiki/C'Thun"
# url = 'https://wowpedia.fandom.com/wiki/Galakrond'


races = []
affiliations = []

for name in tqdm(list(df['Name'])):
    try:
        name = name.split("/")[0].split("_(")[0]
        url = "https://wowpedia.fandom.com/wiki/{}".format(name)
        html = urlopen(url).read()
        soup = BeautifulSoup(html, features="html.parser")

        table =  soup.find("table",{"class":"infobox darktable"})

        text = table.text

        race, affiliation = get_race_and_affiliation(text)

#         print(race)
#         print(affiliation)
        

        races.append(race)
        affiliations.append(affiliation)
    except:
        print(name)
        races.append(0)
        affiliations.append(0)
   

 28%|██████████████████████▏                                                        | 698/2485 [03:19<05:35,  5.33it/s]

Data_Anomaly


 92%|███████████████████████████████████████████████████████████████████████▌      | 2279/2485 [10:16<00:38,  5.32it/s]

Underdev


100%|██████████████████████████████████████████████████████████████████████████████| 2485/2485 [11:08<00:00,  3.72it/s]


In [10]:
df.insert(1, 'Race', races)
df.insert(2, 'Affiliation', affiliations)

In [55]:
# remove the "Warcraft_Lore" and "The_Characters_of_Warcraft" characters

df = df.iloc[[i for i in range(len(df)) if 'Warcraft' not in df.iloc[i]['Name']]]
# reset index
df.reset_index(drop=True, inplace=True)
df

,Name,Race,Affiliation
0,Terokk,Arakkoa,0
1,Talon_King_Ikiss,Arakkoa,Other
2,Archimonde,Man'ari eredar,Other
3,Kil'jaeden,Man'ari eredar,Other
4,Mannoroth,Annihilan,Other
...,...,...,...
2480,the_Jade_Warlord,Mogu,0
2481,the_Jailer,Eternal One,Other
2482,the_Monkey_King,Hozen,Other
2483,the_Stonewright,Venthyr,Other


In [19]:
# remove stuff manually beacuse I am bad a regex

races2 = []
for race in df_lore['Race']:
    if race != 0:
        race = race.replace("Affiliation", "").replace("Race", "").replace("Occupation", "").replace("Reaction", "").replace("Resource", "").replace("Race", "").replace("Class", "").replace("Former affiliation", "").replace("Location","").replace("Level", "")

    races2.append(race)

df_lore['Race'] = races2

In [56]:
with open('df.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)